<a href="https://colab.research.google.com/github/digo-luz/AI-Dev-Scripts/blob/main/C%C3%B3pia_de_GenAI_LLM_Vertex_Exemplo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Nota: Pode ser necessário reiniciar o kernel depois de instalar algum pacote

In [ ]:
from google import genai
from google.genai import types
import base64
import json
import requests

Google Gen AI SDK documentation:  
https://googleapis.github.io/python-genai/

# Autenticação e teste de conectividade

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
#@title Preenchimento do código do projeto
#@markdown Preencha abaixo o código do teu projeto na GCP. <br/>
#@markdown Após preenchimento do código, execute essa célula.   <br/>

project_id = "autobsg"  #@param {type: "string"}
location = "us-central1"  #@param {type: "string"}

llm_model = "gemini-2.0-flash-lite-001" # @param ["gemini-2.0-flash-001","gemini-2.0-flash-lite-001","gemini-1.5-flash-002"] {"allow-input":true}
#@markdown ---
llm_client = genai.Client( vertexai=True, project=project_id, location=location, )

## 01 - Exemplo simples: Hello World

A ) Parâmetros de configuração da chamada do modelo, incluindo instruções sistêmicas sobre a saída do modelo.  
Para saber sobre os parâmetros existentes atualmente, segue link da documentação:  
https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/content-generation-parameters?hl=pt-br

In [ ]:
llm_agent_config = types.GenerateContentConfig(
    candidate_count = 1,
    temperature = 0.6,
    top_p = 1,
    top_k = 40,
    max_output_tokens = 2048,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH, threshold=types.HarmBlockThreshold.BLOCK_NONE),
                       types.SafetySetting(category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT, threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH),
                       types.SafetySetting(category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT, threshold=types.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE),
                       types.SafetySetting(category=types.HarmCategory.HARM_CATEGORY_HARASSMENT, threshold=types.HarmBlockThreshold.OFF),
                       ],
    response_mime_type = "application/json",
    system_instruction=
          [
            'Você é um galanteador brasileiro e deve responder em português, mesmo que seja solicitado para que a resposta seja em outra língua.',
            'Sua missão é fazer piadas de duplo sentido sem conotação sexual.'
          ],
    response_schema = {"type":"OBJECT","properties":{"retorno_do_modelo":{"type":"STRING"}}},
)

B ) Entrada do modelo, também conhecido como request ou request contents.  
Nota, estamos usando uma versão super simplificada com

```
contents = "Me fale sobre os Lusíadas."
```

Por usar apenas parâmetros padrão, a criação desse `contents` seria equivalente a esse:  
```
contents = []
content = types.Content(
    role="user", # Role must be either ‘user’ or ‘model’. Useful to set for multi-turn conversations, otherwise can be left blank or unset. If role is not specified, SDK will determine the role.
    parts=[
        types.Part.from_text(text="""Me fale sobre os Lusíadas.""")
    ]
)
contents.append(content)
```

In [ ]:
contents = 'Me fale sobre os Lusíadas.'

C ) Chama e obtém o retorno do modelo. Usando o SDK, o mesmo vem em um objeto estruturado do tipo `google.genai.types.GenerateContentResponse`

In [ ]:
llm_ret = llm_client.models.generate_content(
  model = llm_model,
  contents = contents,
  config = llm_agent_config,
  )
display(llm_ret)

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='{\n  "retorno_do_modelo": "Ah, os Lusíadas! Uma epopeia que é um verdadeiro \\"tesouro\\" da literatura portuguesa. Sabe, ela é tão \\"profunda\\" que até hoje a gente se \\"afoga\\" em suas palavras. Mas, falando sério, é uma obra que nos leva a \\"navegar\\" por histórias incríveis, cheias de \\"descobertas\\" e \\"aventuras\\"."\n}')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=-0.3107082894507875, finish_reason=<FinishReason.STOP: 'STOP'>, grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=[SafetyRating(blocked=None, category=<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 'HARM_CATEGORY_HATE_SPEECH'>, probability=<HarmProbability.NEGLIGIBLE: 'NEGLIGIBLE'>, probability_score=1.224

In [ ]:
print(llm_ret.text)

{
  "retorno_do_modelo": "Ah, os Lusíadas! Uma epopeia que é um verdadeiro \"tesouro\" da literatura portuguesa. Sabe, ela é tão \"profunda\" que até hoje a gente se \"afoga\" em suas palavras. Mas, falando sério, é uma obra que nos leva a \"navegar\" por histórias incríveis, cheias de \"descobertas\" e \"aventuras\"."
}


## 02 - Exemplo com saída em JSON estruturado.

Sobreescreve o Guard Rail da estrutura de resposta.

In [ ]:
llm_agent_config.response_schema = {
  "type": "object",
  "properties": {
    "n1": {
      "type": "integer"
    },
    "n2": {
      "type": "integer"
    },
    "senna": {
      "type": "string"
    },
    "piada": {
      "type": "string"
    },
    "extra": {
      "type": "string"
    }
  },
  "required": [
    "n1",
    "n2",
    "senna",
    "piada"
  ]
}

Aqui, o `contents` será um prompt simples.

In [ ]:
contents = """Retorne um json contendo o seguinte conteúdo:
- n1: Um número aleatório entre 1 e 1000;
- n2: Um número primo maior que 100;
- senna: A data de nascimento do Ayrton Senna no formato dd-MM-YYYY;
- piada: Um texto com uma piada sobre alunos e professores."""

In [ ]:
llm_ret = llm_client.models.generate_content(
  model = llm_model,
  contents = contents,
  config = llm_agent_config,
  )
display(llm_ret)

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='{\n  "n1": 427,\n  "n2": 101,\n  "piada": "Por que o professor de matemática sempre anda com a cara fechada? Porque ele vive com problemas!",\n  "senna": "21-03-1960"\n}')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=-0.11931957517351423, finish_reason=<FinishReason.STOP: 'STOP'>, grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=[SafetyRating(blocked=None, category=<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 'HARM_CATEGORY_HATE_SPEECH'>, probability=<HarmProbability.NEGLIGIBLE: 'NEGLIGIBLE'>, probability_score=1.0148615e-05, severity=<HarmSeverity.HARM_SEVERITY_NEGLIGIBLE: 'HARM_SEVERITY_NEGLIGIBLE'>, severity_score=None), SafetyRating(blocked=None, category=<HarmCategory.HARM_C

In [ ]:
print(llm_ret.text)

{
  "n1": 427,
  "n2": 101,
  "piada": "Por que o professor de matemática sempre anda com a cara fechada? Porque ele vive com problemas!",
  "senna": "21-03-1960"
}


Conversão de `string` para `json`

In [ ]:
json_ret = json.loads( llm_ret.text )
json_ret

{'n1': 427,
 'n2': 101,
 'piada': 'Por que o professor de matemática sempre anda com a cara fechada? Porque ele vive com problemas!',
 'senna': '21-03-1960'}

## 03 - Exemplo com múltiplas entradas estruturadas

In [ ]:
#Obtém um arquivo da internet e armazena seus bytes
image_url = "https://storage.googleapis.com/ds-publico/IA/llminput01.jpeg"
image_bytes = requests.get(image_url).content
base64_image = base64.b64encode(image_bytes).decode('utf-8')

In [ ]:
contents = [
  types.Content(
    role="user",
    parts=[
      types.Part.from_text(text="""Me conte uma piada sobre brasileiro em português."""),
      types.Part.from_bytes(data=image_bytes, mime_type="image/jpeg",),
      types.Part.from_text(text="""Considere a imagem fornecida para a piada.""")
    ]
  )
]

In [ ]:
llm_ret = llm_client.models.generate_content(
  model = llm_model,
  contents = contents,
  config = llm_agent_config,
  )
print(llm_ret.text)

{
  "n1": 50,
  "n2": 50,
  "piada": "O que o brasileiro foi fazer na escada? Subir na vida!",
  "senna": "Ayrton Senna foi o melhor piloto de todos os tempos.",
  "extra": "O Brasil é um país de gente feliz!"
}


# EXERCICIO

A URL a seguir contém o relatório de desempenho do quarto trimestre de 2024 da Petrobrás. Por ser o último trimestre, também é um relatório anual.  


URL: https://storage.googleapis.com/ds-publico/IA/petrobras_ri_2024T4_release.pdf


Você é um analista do mercado financeiro e deve analisar o mais rápido possível este relatório trimestral / anual.  

Considere que deverá retornar um arquivo JSON com a seguinte estrutura exemplo:

```
{
  "resumo": Resumo do relatório;
  "opiniao": Opinião sobre o relatório;
  "classificacao": positiva, negativa ou neutra sobre o relatório;
  "dividendos": Se houver, mencionar os dividendos (opcional);
  "faturamento": Se houver, mencionar o faturamento (opcional);
  "lucro": Se houver, mencionar o lucro (opcional);
  "extra": Alguma informação relevante (opcional)
}

```


Considerações específicas da sua análise:
- O Mercado espera uma comparação entre trimestres mais importante do que a comparação anual;
- Considere que houve troca de presidente da companhia na opinião fornecida;

Se solicitado, não se esqueça de submeter o Notebook com sua resposta pelo Portal

------

In [ ]:
llm_agent_config.response_schema = {
  "type": "object",
  "properties": {
    "resumo": {
      "type": "string"
    },
    "opiniao": {
      "type": "string"
    },
    "classificacao": {
      "type": "string"
    },
    "dividendos": {
      "type": "string"
    },
    "faturamento": {
      "type": "number"
    },
    "lucro": {
      "type": "number"
    },
    "extra": {
      "type": "string"
    }
  },
  "required": [
    "resumo",
    "opiniao",
    "classificacao",
    "dividendos",
    "faturamento",
    "lucro",
    "extra"
  ]
}

In [ ]:
contents = """
Por favor, analise o relatório, considere o documento fornecido para relatório.

Retorne um JSON com a seguinte estrutura:

{
  "resumo": "Um resumo conciso do relatório, destacando os pontos principais.",
  "opiniao": "Sua opinião sobre o desempenho da Petrobrás neste trimestre, considerando a troca de presidente e o foco do mercado em comparações trimestrais.",
  "classificacao": "Uma classificação geral do relatório: 'positiva', 'negativa' ou 'neutra'.",
  "dividendos": "Informações sobre dividendos anunciados, se houver.",
  "faturamento": "O valor do faturamento da Petrobrás neste trimestre, se disponível.",
  "lucro": "O valor do lucro líquido da Petrobrás neste trimestre, se disponível.",
  "extra": "Quaisquer outras informações relevantes ou observações que você considere importantes."
}

Lembre-se de:

* Priorizar a comparação entre o desempenho deste trimestre e o trimestre anterior.
* Considerar a mudança na presidência da companhia em sua análise.
* As informações sobre dividendos, faturamento e lucro são opcionais, inclua-as apenas se estiverem disponíveis no relatório.
"""

In [ ]:
#Obtém um arquivo da internet e armazena seus bytes
image_url = "https://storage.googleapis.com/ds-publico/IA/petrobras_ri_2024T4_release.pdf"
image_bytes = requests.get(image_url).content
# base64_image = base64.b64encode(image_bytes).decode('utf-8')

In [ ]:
contents = [
  types.Content(
    role="user",
    parts=[
      types.Part.from_text(text="""Por favor, analise o relatório."""),
      types.Part.from_bytes(data=image_bytes, mime_type="application/pdf",),
      types.Part.from_text(text="""considere o documento fornecido para relatório.""")
    ]
  )
]

In [ ]:
llm_ret = llm_client.models.generate_content(
  model = llm_model,
  contents = contents,
  config = llm_agent_config,
  )
print(llm_ret.text)

{
  "classificacao": "Relatório de Desempenho 2024 da Petrobras",
  "dividendos": "A Petrobras distribuiu R$ 102,6 bilhões em dividendos, demonstrando seu compromisso com os investidores.",
  "extra": "O relatório destaca o excelente desempenho financeiro e operacional da Petrobras em 2024, com ênfase na geração de caixa e na redução da dívida.",
  "faturamento": 490.829,
  "lucro": 36.606,
  "opiniao": "O relatório apresenta uma visão positiva do desempenho da Petrobras, com foco na geração de valor e no crescimento sustentável.",
  "resumo": "O Relatório de Desempenho 2024 da Petrobras apresenta os resultados financeiros e operacionais da empresa, destacando a geração de caixa, a redução da dívida e os investimentos em novos projetos."
}
